In [62]:
import geopandas as gpd
import matplotlib.pyplot as plt
import requests
import pandas as pd
import urllib.request, json

In [63]:
def API_Request(Stations, SubscriptionKey):
    
    #API Headers
    url = f"https://gateway.apiportal.ns.nl/Spoorkaart-API/api/v1/traject.geojson?stations={Stations}"
    headers = {
    'Cache-Control': 'no-cache',
    'Ocp-Apim-Subscription-Key': SubscriptionKey,
    }

    # Make the API request
    req = urllib.request.Request(url, headers=headers)
    response = urllib.request.urlopen(req)
    response_content = response.read().decode('utf-8')

    # Load the response content into JSON
    geojson_data = json.loads(response_content)

    # Convert the GeoJSON data to a GeoDataFrame
    gdf = gpd.GeoDataFrame.from_features(geojson_data['features'])

    return gdf


In [80]:
# function to make a dictionary with the seats form the transervices csv
def SeatSorter(TrainServicesData, TrainTravelData):
    # Inladen CSV met alle NS treindiensten
    TrainServices = pd.read_csv(TrainServicesData, delimiter = ';')

    # Dictionary maken met per treinserie een dataframe met ieder stukje tussen stations er in
    dataframes_dict = {}

    TrainServices.loc[0,'Code']
    TrainServices.loc[0,'String'].split(',')

    for i in range(len(TrainServices)):
        a = TrainServices.loc[i,'String'].split(',')
        df = pd.DataFrame({'From':[],'To':[],'Seats':[]})
        for j in range(len(a) - 1):
            new_row = pd.DataFrame({'From': [a[j]], 'To': [a[j + 1]], 'Seats':[None]})
            df = pd.concat([df, new_row], ignore_index=True)
        dataframes_dict[TrainServices.loc[i,'Code']] = df

        a = TrainServices.loc[i,'String'].split(',')
        a = a[::-1]
        df = pd.DataFrame({'From':[],'To':[],'Seats':[]})
        for j in range(len(a) - 1):
            new_row = pd.DataFrame({'From': [a[j]], 'To': [a[j + 1]], 'Seats':[None]})
            df = pd.concat([df, new_row], ignore_index=True)
        dataframes_dict[TrainServices.loc[i,'Code'] + 1] = df

    # Dictionary vullen met stoel aantallen
    for i in range(len(TrainTravelData)):
        if TrainTravelData.loc[i, 'JourneyNumber'] % 2 != 0:
            a = TrainTravelData.loc[i, 'JourneyNumber']//100
            c = 0
            for j in range(len(dataframes_dict[a*100+ 1])):
                if TrainTravelData.loc[i, 'UserStopCodeBegin'] == dataframes_dict[a*100+ 1].loc[j, 'From'].upper():
                    c = 1
                if c == 1:
                    if dataframes_dict[a*100 + 1].loc[j, 'Seats'] == None:
                        dataframes_dict[a*100+ 1].loc[j, 'Seats'] = TrainTravelData.loc[i, 'Seats']
                    else:
                        dataframes_dict[a*100+ 1].loc[j, 'Seats'] += TrainTravelData.loc[i, 'Seats']
                if TrainTravelData.loc[i, 'UserStopCodeEnd'] == dataframes_dict[a*100+ 1].loc[j, 'To'].upper():
                    break

        else:
            a = TrainTravelData.loc[i, 'JourneyNumber']//100
            c = 0
            for j in range(len(dataframes_dict[a*100])):
                if TrainTravelData.loc[i, 'UserStopCodeBegin'] == dataframes_dict[a*100].loc[j, 'From'].upper():
                    c = 1
                if c == 1:
                    if dataframes_dict[a*100].loc[j, 'Seats'] == None:
                        dataframes_dict[a*100].loc[j, 'Seats'] = TrainTravelData.loc[i, 'Seats']
                    else:
                        dataframes_dict[a*100].loc[j, 'Seats'] += TrainTravelData.loc[i, 'Seats']
                if TrainTravelData.loc[i, 'UserStopCodeEnd'] == dataframes_dict[a*100].loc[j, 'To'].upper():
                    break

    return dataframes_dict

In [65]:
# function to assign a color to a number based on where in the range of numbers it lies
def interpolate_color(lower_limit, upper_limit, lower_color, upper_color, number):
    # Clamp the number between the lower and upper limit
    number = max(min(number, upper_limit), lower_limit)
    
    # Calculate the interpolation factor
    factor = (number - lower_limit) / (upper_limit - lower_limit)
    
    # Interpolate each RGB component
    interpolated_color = tuple(
        int(lower_component + (upper_component - lower_component) * factor)
        for lower_component, upper_component in zip(lower_color, upper_color)
    )
    
    #normalize the color for usage
    color = [interpolated_color / 255.0 for interpolated_color in interpolated_color] 

    return color

In [66]:
#   NS Bron aanpassen
# Inladen CSV en toevoegen kolommen voor Stoelen en Bezette stoelen
TravelData = pd.read_csv('OC_NS_20241007.csv')
TravelData['Seats'] = 0
TravelData['Occupied Seats'] = 0

# Voor iedere treinserie het aantal stoelen toevoegen
for i in range(len(TravelData)):
    if TravelData.loc[i, 'VehicleType'] == "VIRM":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 100
    if TravelData.loc[i, 'VehicleType'] == "DDZ":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 100
    if TravelData.loc[i, 'VehicleType'] == "FLIRT FFF":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 53
    if TravelData.loc[i, 'VehicleType'] == "ICM":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 75
    if TravelData.loc[i, 'VehicleType'] == "ICNG25":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 52
    if TravelData.loc[i, 'VehicleType'] == "SLT":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 54
    if TravelData.loc[i, 'VehicleType'] == "SNG":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 50
    if TravelData.loc[i, 'VehicleType'] == "SW7-25KV":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 48
    if TravelData.loc[i, 'VehicleType'] == "SW9-25KV":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 48

# Maar 1 week behouden dus 3 dagen weghalen uit de tabel
TravelData['OperatingDay'] = pd.to_datetime(TravelData['OperatingDay'])
dates_to_exclude = pd.to_datetime(['2024-10-14', '2024-10-15', '2024-10-16'])
df_filtered = TravelData[~TravelData['OperatingDay'].isin(dates_to_exclude)]
TravelData = df_filtered.reset_index(drop=True)

#treincodes boven 700000 aanpassen
for i in range(len(TravelData)):
    if TravelData.loc[i, 'JourneyNumber'] > 700000:
        TravelData.loc[i, 'JourneyNumber'] -= 700000

#treincode tussen 200000 en 700000 aanpassen
for i in range(len(TravelData)):
    if 200000 < TravelData.loc[i, 'JourneyNumber'] < 700000:
        TravelData.loc[i, 'JourneyNumber'] -= 200000

display(TravelData)

,DataOwnerCode,OperatingDay,LinePlanningNumber,JourneyNumber,ReinforcementNumber,TimingLinkOrder,UserStopCodeBegin,UserStopCodeEnd,Occupancy,VehicleType,TotalNumberOfCoaches,Seats,Occupied Seats
0,NS,2024-10-11,NaN,6649,0,14,HT,TB,2,FLIRT FFF,7,371,0
1,NS,2024-10-11,NaN,786,0,6,ASDZ,SHL,3,ICM,4,300,0
2,NS,2024-10-10,NaN,3342,0,2,LAA,GVM,2,SNG,6,300,0
3,NS,2024-10-12,NaN,7367,0,8,MRN,VNDW,2,SNG,4,200,0
4,NS,2024-10-11,NaN,4674,0,1,ASD,ASS,2,SNG,6,300,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
212902,NS,2024-10-07,NaN,6369,0,1,HLM,HAD,2,SLT,6,324,0
212903,NS,2024-10-09,NaN,3550,0,1,VL,BR,2,VIRM,6,600,0
212904,NS,2024-10-07,NaN,7572,0,3,WF,ED,2,FLIRT FFF,3,159,0
212905,NS,2024-10-11,NaN,2284,0,5,LEDN,LAA,3,VIRM,6,600,0


In [81]:
a = SeatSorter('TrainServices.csv', TravelData)

{np.int64(500):     From    To  Seats
0     Gn  Gerp  60650
1   Gerp   Hrn  60650
2    Hrn   Asn  57925
3    Asn    Bl  61100
4     Bl   Hgv  61100
5    Hgv    Mp  61100
6     Mp    Zl  61625
7     Zl    Wz  67750
8     Wz   Hde  67750
9    Hde    Ns  67750
10    Ns    Hd  67750
11    Hd   Eml  67750
12   Eml    Pt  67750
13    Pt   Nkk  67750
14   Nkk  Avat  67750
15  Avat  Amfs  67750
16  Amfs   Amf  67750
17   Amf   Dld  50225
18   Dld   Bhv  50225
19   Bhv   Uto  50225
20   Uto    Ut  50225
21    Ut  Utlr  55250
22  Utlr   Utt  55250
23   Utt   Vtn  55250
24   Vtn    Wd  55250
25    Wd   Gdg  55250
26   Gdg    Gd  55250
27    Gd   Nwk  55250
28   Nwk   Cps  55250
29   Cps   Rta  55250
30   Rta   Rtn  52925
31   Rtn   Rtd  52925, np.int64(501):     From    To  Seats
0    Rtd   Rtn  54725
1    Rtn   Rta  54725
2    Rta   Cps  53525
3    Cps   Nwk  53525
4    Nwk    Gd  53525
5     Gd   Gdg  55350
6    Gdg    Wd  55350
7     Wd   Vtn  55350
8    Vtn   Utt  55350
9    Utt  Utlr  55350


In [95]:
display(a[901])

,From,To,Seats
0,Bd,Bdpb,45132
1,Bdpb,Rlb,45132
2,Rlb,Rtst,45132
3,Rtst,Rtz,45132
4,Rtz,Rtb,45132
5,Rtb,Rtd,45132
6,Rtd,Hfd,52216
7,Hfd,Shl,52216
8,Shl,Asdl,52216
9,Asdl,Ass,52216
